In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
from datetime import datetime
import lightgbm as lgb

warnings.filterwarnings('ignore')

519it [54:49,  6.34s/it]


In [ ]:
dtype={
    1:'category',
    2:"float32",
    4:"uint16",
    8:"float32",
    9:"float32",
    10:"float32",
    11:"float32",
    12:"uint8",
    13:"uint64",
}
from tqdm import tqdm
def get_df(f):
    files = []
    for chunk in tqdm(pd.read_csv(f, sep='\|', header=None, usecols=[1, 2, 4, 8, 9, 10, 11, 12, 13], chunksize=1000000, dtype=dtype)):
        files.append(chunk)
    return files
chunk = get_df('E:\cp_rawdata_0509.txt')

df = pd.concat(chunk,axis=0)
del chunk

In [2]:
col_names = ['uin', 'kill_time', 'index',
             'pitch', 'yaw','pitch_r', 'yaw_r', 'type', 'weapon_id']
df.columns = col_names
df['uin'] = df['uin'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518451376 entries, 0 to 518451375
Data columns (total 9 columns):
 #   Column     Dtype   
---  ------     -----   
 0   uin        category
 1   kill_time  float32 
 2   index      uint16  
 3   pitch      float32 
 4   yaw        float32 
 5   pitch_r    float32 
 6   yaw_r      float32 
 7   type       uint8   
 8   weapon_id  uint64  
dtypes: category(1), float32(5), uint16(1), uint64(1), uint8(1)
memory usage: 16.9 GB


In [3]:
df_ = pd.read_csv('label.csv')
pos_uin = list(df_[df_['label'] == 1]['uin'])
neg_uin = list(df_[df_['label'] == 0]['uin'])
sample_uin = pos_uin + neg_uin
df = df[df['uin'].isin(sample_uin)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206786069 entries, 3 to 518451375
Data columns (total 9 columns):
 #   Column     Dtype   
---  ------     -----   
 0   uin        category
 1   kill_time  float32 
 2   index      uint16  
 3   pitch      float32 
 4   yaw        float32 
 5   pitch_r    float32 
 6   yaw_r      float32 
 7   type       uint8   
 8   weapon_id  uint64  
dtypes: category(1), float32(5), uint16(1), uint64(1), uint8(1)
memory usage: 8.3 GB


In [5]:
df.sort_values(['uin', 'kill_time', 'index'], inplace=True)
df.to_pickle('train_total.pkl')